## RAG
- hallucination 환각 문제 발생 
- 거대언어의 문제점: 최신 정보의 부재
- -> 극복방안: 분야의 제한없이 광범위한 주제의 질문을 받고 답변하는 질의응답 시스템을 응용해서 RAG
- 거대언어 이전: Q&A쌍을 만들어 둠. 
  - 쿼리 들어오면 제일 유사한 쿼리를 찾아냄. ex) 코사인 유사도이용 후 tf-idf를 이용한 벡터형으로 변환하여 저장.

### ODQA(Open Domain Query Answering)
  - 1단계: 위키피디어와 같은 대량의 문서들을 수집하고 인덱싱해서 지식베이스에 저장.최근에는 딥러닝기법을 이용해 각 문서를 벡터로 임베딩. 문서간 유사도가 잘 표현할 수 있도록 임베딩 방법을 학습.
  - 2단계: 질의가 들어오면 임베딩하고 지식베이스에 임베딩된 문서들과 유사도를 비교해서 응답이 있을만한 문서를 검색.
  - 3단계: 질의와 검색된 문서를 학습된 언어모델에 입력하고 문서로부터 응답을 추출. 

### RAG(Retrieval-Augmented Generation): 검색된 정보를 바탕으로 언어 모델이 텍스트를 생성하는 방식으로 작동
   - 수집한 데이터를 일정 단위(문서 혹은 그보다 작은단위)로 임베딩(Document Indexing).
   - 쿼리가 들어오면 임베딩하고 수집된 데이터 임베딩과 유사도를 비교해 가장 유사한 데이터 검색(Retriever).
   - 쿼리 + 데이터(문서)로 프롬프트를 작성해 LLM으로부터 결과를 받음(Generator).

## RAG 구성 요소
- Document loaders: 대상 문서 읽어오기
- Text Splitting: 문서를 chunk 단위로 splitting
- Text Embedding: 벡터로 임베딩
- Vector Stores: 벡터(+원본) 저장
- Retrievers: 쿼리에 대해 유사벡터 / 문서검색 및 반환

In [1]:
import os
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

#### 문서 loader 

In [9]:
import requests
from langchain.document_loaders import TextLoader

# 문서들을 읽어오기 위한 라이브러리. 이후 처리를 위해 문자열의 리스트 형태로 읽어들임
# LangChain은 부가적으로 필요한 메타정보를 추가하여 Document객체로 저장

#url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/ modules/state_of_the_union.txt"
#res = requests.get(url)
#with open("state_of_the_union.txt", "w") as f:
#    f.write(res.text)
    
loader = TextLoader('state_of_the_union.txt')
documents = loader.load() # Document 객체 리스트를 반환

RuntimeError: Error loading state_of_the_union.txt

In [7]:
print(len(documents[0].page_content))
print(documents[0].page_content[:200])

14
404: Not Found


#### text splitting 종류 
- Character Splitting: 문서를 n개의 문자 단위로 split.
- Recursive Character Text Splitting: chunk size를 넘지 않는 범위에서 separator 기준으로 분리.
- Document Specific Splitting: Level 2를 기반으로 하여, 문서의 종류에 따라 특성에 맞게 각기 다른 separator 적용.
- Semantic Splitting: 하나의 Chunk가 의미적으로 최대한 유사하도록 split. 유사도를 계산함.
- Agentic Splitting: 첫 문장의 주제 파악 후 현재 문장의 주제를 파악. 이전 문장의 주제와 비교해서 동일하면 chunk 유지하며 진행. 주제가 달라지면 새로운 chunk 생성. 2로 돌아가서 마지막 문장까지 반복
- 보통 2를 제일 많이 사용하는 듯? 

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# RecursiveCharacterTextSplitter: 객체 생성, 청크의 최대 크기(500), 청크가 겹치는 부분의 크기(20) 지정

recursive_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,
                                                    chunk_overlap = 20)

# 텍스트 분할 

chunks = recursive_splitter.split_text(documents[0].page_content)

print(len(chunks)) # chunk 개수 확인

1


In [12]:
# 각 청크의 크기 확인 

chunk_sizes = [len(chunk) for chunk in chunks]

print(max(chunk_sizes), min(chunk_sizes), chunk_sizes[:10])

14 14 [14]


In [13]:
docs = recursive_splitter.split_documents(documents)
print(len(docs))
print(docs)

1
[Document(page_content='404: Not Found', metadata={'source': './state_of_the_union.txt'})]


#### Text Embedding
- 텍스트 데이터를 숫자 벡터로 변환

In [32]:
# from langchain_openai import OpenAIEmbeddings: OpenAIEmbeddings를 이용해 두개의 텍스트를 임베딩하고 결과 확인

from langchain_community.embeddings import OpenAIEmbeddings

embed = OpenAIEmbeddings(model = 'text-embedding-ada-002',
                         openai_api_key = OPENAI_API_KEY
                        )

texts = ['this is the first chunk of text','then another second chunk of text is here']

res = embed.embed_documents(texts) # 문서 임베딩 결과 저장

print(len(res), len(res[0]), len(res[1]), res[0][:10])

2 1536 1536 [0.0032154050918757333, -0.00967098752427458, -0.009704016878696898, -0.03255370564160612, 0.00018114522014075682, 0.026582003949986163, -0.013859105939734414, -0.006737983179879034, -0.021019864390557856, -0.03622656612807778]


In [34]:
# Sentence Transformers on Hugging Face: BERT를 기반으로 하여 Sentence를 임베딩하도록 학습한 모델

from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embed = SentenceTransformerEmbeddings(model_name = "all-MiniLM-L6-v2")

texts = ['this is the first chunk of text', 'then another second chunk of text is here']

res = embed.embed_documents(texts) # 문서 임베딩 결과 저장 

print(len(res), len(res[0]), len(res[1]), res[0][:10])

C:\Users\82103\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  pass
C:\Users\82103\anaconda3\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\82103\anaconda3\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\82103\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2 384 384 [-0.011914781294763088, 0.0891803726553917, 0.03824286907911301, 0.01908119209110737, 0.059775590896606445, 0.0053163859993219376, 0.03878144174814224, -0.008798955008387566, 0.06023487076163292, -0.015470323152840137]


### Vector Store
- 임베딩 벡터 저장
- query에 대해 가장 유사한 벡터를 반환하는 retrieve 기능을 함께 제공
- query와 유사한 문서의 내용을 store에 검색해서 이를 바탕으로 답변을 생성

In [35]:
# chroma 라이브러리 이용 - 벡터 스토어 생성 및 관리, 유사도 검색, 확장성
import chromadb

chroma_client = chromadb.Client() # 클라이언트 생성 

collection = chroma_client.create_collection(name = "my_collection") # collection: 벡터를 저장하고 관리하는 논리적 단위 생성.

# 문서 추가 
collection.add(
    documents = ["This is a document", "This is another document"],
    metadatas = [{"source": "my_source1"}, {"source": "my_source2"}],
    ids = ["id1", "id2"]
)

# collection query: 유사한 문서 검색
results = collection.query( query_texts=["This is a query document"],
                           n_results=2
                          )
print(results)

C:\Users\82103\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|█████| 79.3M/79.3M [00:22<00:00, 3.70MiB/s]


{'ids': [['id1', 'id2']], 'distances': [[0.7111214399337769, 1.0109773874282837]], 'metadatas': [[{'source': 'my_source1'}, {'source': 'my_source2'}]], 'embeddings': None, 'documents': [['This is a document', 'This is another document']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


- ids: 쿼리된 문서들의 고유 ID를 포함하는 리스트
- distances: 쿼리된 문서들과 입력 쿼리 문서 간의 유사도 점수(거리). 거리 값이 작을수록 두 문서가 유사함
- metadatas: 쿼리된 문서들의 메타데이터
- embeddings: 쿼리된 문서들의 임베딩 벡터를 포함하는 리스트
- documents: 쿼리된 문서들의 실제 텍스트 내용을 포함하는 리스트
- uris: 문서들의 URI(Uniform Resource Identifier)를 포함하는 리스트
- data: 추가적인 데이터를 포함하는 리스트
- included: 쿼리 결과에 포함된 필드를 나타내는 리스트

In [38]:
# state_of_the_union예제
collection = chroma_client.create_collection(name = "state_of_the_union")

ids = ['id'+str(i) for i in range(len(docs))]
doc_texts = [docs[i].page_content for i in range(len(docs))]
doc_metadatas = [docs[i].metadata for i in range(len(docs))]
collection.add(documents = chunks,
               metadatas = doc_metadatas,
               ids = ids
              )
# Query the collection
results = collection.query(
    query_texts = ["What did the president say about Ketanji Brown Jackson"],
    n_results = 1
)
print(results)

{'ids': [['id0']], 'distances': [[1.7565205097198486]], 'metadatas': [[{'source': './state_of_the_union.txt'}]], 'embeddings': None, 'documents': [['404: Not Found']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


In [39]:
# LangChain의 Chroma 라이브러리 사용 

from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(docs, embed) 
query = "What did the president say about Ketanji Brown Jackson"
query_result = db.similarity_search(query)
print(len(query_result), query_result[0])

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


1 page_content='404: Not Found' metadata={'source': './state_of_the_union.txt'}


- Chroma와 같은 벡터 스토어를 사용할 때, 데이터를 로드할 때 임베딩 함수를 지정해야 한다는 것은,텍스트 데이터를 벡터로 변환하는 함수가 필요하다는 의미

In [40]:
# Chroma 벡터 스토어 생성 및 데이터 저장 
db2 = Chroma.from_documents(docs, embed, persist_directory = "./chroma_db")

# 디스크에서 벡터 스토어 로드
db3 = Chroma(persist_directory = "./chroma_db", embedding_function = embed)

# 유사도 검색 수행
docs = db3.similarity_search(query)
print(docs[0])

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


page_content='404: Not Found' metadata={'source': './state_of_the_union.txt'}


### Retriever를 이용한 Q&A 구현
- Retriever
  - 주어진 query를 벡터로 변환하고 vector store에서 유사도가 높은 벡터들을 반환.
  - 일반적으로 vector store가 기능을 함께 제공.
- Q&A 구현
  - Retriever가 반환한 Context와 원래 query로 프롬프트를 생성하고 이를 LLM에 입력하여 답변을 생성.
  - Context의 내용만 이용해 답변하도록 하는 프롬프트 템플릿이 필요함.

In [41]:
from langchain import PromptTemplate

# Context만 이용해서 답변하도록 프롬프트 템플릿 생성

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".
Context: {context}
Question: {query}
Answer: 
"""

prompt_template = PromptTemplate(
    input_variables = ["context", "query"],
    template = template
)

- 추가로 LangChain hub: LLM의 효율적인 관리를 위해 제공되는 라이브러리
  - RetrievalQA Chain: RAG 파이프라인 예제들
  - Runnable PromptTemplate: 프롬프트 템플릿 예제들

In [79]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt") # langchain hub에 저장된 rag prompt 사용
print(prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


- RAG를 위한 Chain 생성

In [78]:
# from langchain_openai import ChatOpenAI
# from langchain.schema.runnable import RunnablePassthrough

from langchain_community.chat_models import ChatOpenAI
# from langchain_core.runnables import RunnableLambda
from langchain.schema.runnable import RunnablePassthrough

retriever = db.as_retriever() # vector store를 retriever로 사용
openai = ChatOpenAI(
    model_name ='gpt-3.5-turbo',
    api_key = OPENAI_API_KEY
)

rag_chain = ({"context": retriever, "query": RunnablePassthrough()}
             |prompt_template
             |openai
            )

query = "What did the president say about Justice Breyer"
rag_chain.invoke(query)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


AIMessage(content="I don't know.", response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 76, 'total_tokens': 81}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dffa9f30-4499-45de-88e9-090947f5f9d4-0')

- RunnableParallel: 두 개 이상의 Runnable 이 (sequential이아닌) parallel하게 실행되어야 하는 상황에 사용

In [77]:
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda
from langchain.schema.runnable import RunnablePassthrough

setup_and_retrieval = RunnableParallel({"context":retriever,"question":RunnablePassthrough()})
chain = setup_and_retrieval | prompt | model | output_parser

# sequence = runnable_1 | runnable_2
# 혹은 sequence = RunnableSequence(first=runnable_1, last=runnable_2)

NameError: name 'model' is not defined

- 출력이 AIMessage 객체가 아닌 문자열이 되도록 변경

In [74]:
from langchain.schema.output_parser import StrOutputParser

retriever = db.as_retriever()

rag_chain=({"context": retriever, "query": RunnablePassthrough()}
           |prompt_template
           |openai
           |StrOutputParser()
          )
query = "What did the president say about Justice Breyer"
rag_chain.invoke(query)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


"I don't know."

- 프롬프트 템플릿 대신 RetrievalQA 사용

In [75]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm = openai,
    chain_type = "stuff",
    retriever = db.as_retriever()
)

query ="What did the president say about Justice Breyer"
qa.invoke(query)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


{'query': 'What did the president say about Justice Breyer',
 'result': "I'm sorry, but I do not have the specific information about what the president said about Justice Breyer."}

- RetrievalQAWithSourcesChain 사용

In [76]:
from langchain.chains import RetrievalQAWithSourcesChain
qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm = openai,
    chain_type = "stuff",
    retriever = db.as_retriever()
)
query = "What did the president say about Justice Breyer"
qa_with_sources.invoke(query)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


{'question': 'What did the president say about Justice Breyer',
 'answer': "I don't know.\n",
 'sources': ''}